### Nonlocal Scopes

Functions defined inside anther function can reference variables from that enclosing scope, just like functions can reference variables from the global scope.

In [1]:
def outer_func():
    x = 'hello'
    
    def inner_func():
        print(x)
    
    inner_func()

In [2]:
outer_func()

hello


In fact, any level of nesting is supported since Python just keeps looking in enclosing scopes until it finds what it needs (or fails to find it by the time it finishes looking in the built-in scope, in which case a runtime error occurrs.)

In [3]:
def outer_func():
    x = 'hello'
    def inner1():
        def inner2():
            print(x)
        inner2()
    inner1()

In [4]:
outer_func()

hello


But if we **assign** a value to a variable, it is considered part of the local scope, and potentially **masks** enclsogin scope variable names:

In [5]:
def outer():
    x = 'hello'
    def inner():
        x = 'python'
    inner()
    print(x)

In [6]:
outer()

hello


As you can see, **x** in **outer** was not changed.

To achieve this, we can use the **nonlocal** keyword:

In [9]:
def outer():
    x = 'hello'
    def inner():
        nonlocal x
        x = 'python'
    inner()
    print(x)

In [10]:
outer()

python


Of course, this can work at any level as well:

In [11]:
def outer():
    x = 'hello'
    
    def inner1():
        def inner2():
            nonlocal x
            x = 'python'
        inner2()
    inner1()
    print(x)

In [12]:
outer()

python


How far Python looks up the chain depends on the first occurrence of the variable name in an enclosing scope.

Consider the following example:

In [19]:
def outer():
    x = 'hello'
    def inner1():
        x = 'python'
        def inner2():
            nonlocal x
            x = 'monty'
        print('inner1 (before):', x)
        inner2()
        print('inner1 (after):', x)
    inner1()
    print('outer:', x)

In [20]:
outer()

inner1 (before): hello
inner1 (after): monty
outer: monty


What happened here, is that `x` in `inner1` **masked** `x` in `outer`. But `inner2` indicated to Python that `x` was nonlocal, so the first local variable up in the enclosing scope chain Python found was the one in `inner1`, hence `x` in `inner2` is actually referencing `x` that is local to `inner1`

We can change this behavior by making the variable `x` in `inner` nonlocal as well:

In [21]:
def outer():
    x = 'hello'
    def inner1():
        nonlocal x
        x = 'python'
        def inner2():
            nonlocal x
            x = 'monty'
        print('inner1 (before):', x)
        inner2()
        print('inner1 (after):', x)
    inner1()
    print('outer:', x)

In [22]:
outer()

inner1 (before): python
inner1 (after): monty
outer: monty


In [23]:
x = 100
def outer():
    x = 'python'  # masks global x
    def inner1():
        nonlocal x  # refers to x in outer
        x = 'monty' # changed x in outer scope
        def inner2():
            global x  # refers to x in global scope
            x = 'hello'
        print('inner1 (before):', x)
        inner2()
        print('inner1 (after):', x)
    inner1()
    print('outer', x)    

In [24]:
outer()
print(x)

inner1 (before): monty
inner1 (after): monty
outer monty
hello


But this will not work. In `inner` Python is looking for a local variable called `x`. `outer` has a label called `x`, but it is a global variable, not a local one - hence Python does not find a local variable in the scope chain.

In [33]:
x = 100
def outer():
    global x
    x = 'python'
    print(locals())
    def inner():
        nonlocal x
        x = 'monty'
        print(locals())
    inner()

SyntaxError: no binding for nonlocal 'x' found (<ipython-input-33-4c66cfe1e087>, line 7)

In [30]:
outer()

{}
{}


In [31]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "def outer_func():\n    x = 'hello'\n    \n    def inner_func():\n        print(x)\n    \n    inner_func()",
  'outer_func()',
  "def outer_func():\n    x = 'hello'\n    def inner1():\n        def inner2():\n            print(x)\n        inner2()\n    inner1()",
  'outer_func()',
  "def outer():\n    x = 'hello'\n    def inner():\n        x = 'python'\n    inner()\n    print(x)",
  'outer()',
  "def outer():\n    x = 'hello'\n    def inner():\n        nonlocal x\n        x = 'python'\n    inner()\n    print(x)",
  'outer()',
  "def outer():\n    x = 'hello'\n    def inner():\n        nonlocal x\n        x = 'python'\n    inner()\n    print(x)",
  'outer()',
  "def outer():\n    x = 'hello'\n    \n    def inner1(

In [32]:
x

'monty'

In [53]:
def out():
    a = 2
    def inn(a,b):
        print(a+b)
    return inn
    

In [54]:
a = out()

In [56]:

a(10,12)

22


In [61]:
class out:
    a = 2
    def inn(self,a,b):
        return a+b

In [62]:
a = out()

In [63]:
a.inn

<bound method out.inn of <__main__.out object at 0x7f3cbbdaf400>>

In [64]:
a.inn(12,10)

22